<a href="https://colab.research.google.com/github/noamgafni/STA160Project/blob/main/STA160_Project_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import files
import os, json

uploaded = files.upload()

os.makedirs('/root/.kaggle', exist_ok=True)
os.replace('kaggle.json', '/root/.kaggle/kaggle.json')
os.chmod('/root/.kaggle/kaggle.json', 0o600)

with open('/root/.kaggle/kaggle.json','r') as f:
    creds = json.load(f)
os.environ['KAGGLE_USERNAME'] = creds['username']
os.environ['KAGGLE_KEY'] = creds['key']

print("Kaggle API set up ✔")


Saving kaggle.json to kaggle.json
Kaggle API set up ✔


In [4]:
!pip -q install kagglehub
import kagglehub, os, glob

DS = "isaaclopgu/cryptocurrency-historical-prices-top-100-2025"
path = kagglehub.dataset_download(DS)

print("Path to dataset files:", path)
print("Sample files:")
for p in glob.glob(os.path.join(path, "**/*"), recursive=True)[:10]:
    print("  ", p)


100%|██████████| 12.4M/12.4M [00:01<00:00, 7.47MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/isaaclopgu/cryptocurrency-historical-prices-top-100-2025/versions/66
Sample files:
   /root/.cache/kagglehub/datasets/isaaclopgu/cryptocurrency-historical-prices-top-100-2025/versions/66/Crypto_historical_data.csv


In [5]:
import pandas as pd, glob, os

csv_paths = glob.glob(os.path.join(path, "**/*.csv"), recursive=True)
print("CSV count:", len(csv_paths))
pd.read_csv(csv_paths[0]).head()


CSV count: 1


,Date,Open,High,Low,Close,Volume,ticker,name
0,2020-04-10 00:00:00+00:00,0.832005,1.313487,0.694187,0.951054,87364276,SOL-USD,Solana
1,2020-04-11 00:00:00+00:00,0.951054,1.049073,0.765020,0.776819,43862444,SOL-USD,Solana
2,2020-04-12 00:00:00+00:00,0.785448,0.956670,0.762426,0.882507,38736897,SOL-USD,Solana
3,2020-04-13 00:00:00+00:00,0.890760,0.891603,0.773976,0.777832,18211285,SOL-USD,Solana
4,2020-04-14 00:00:00+00:00,0.777832,0.796472,0.628169,0.661925,16747614,SOL-USD,Solana


In [6]:
import pandas as pd, numpy as np, re

def symbol_from_filename(p):
    b = os.path.basename(p)
    s = re.sub(r'\.csv$','', b)
    s = re.sub(r'[^A-Za-z0-9]','', s)
    return s.upper()

def load_and_standardize(p):
    df = pd.read_csv(p)
    df.columns = [c.strip().lower() for c in df.columns]

    rename = {}
    for c in df.columns:
        if c in ["date","datetime","timestamp","time"]: rename[c] = "timestamp"
        elif c in ["open","o"]: rename[c] = "open"
        elif c in ["high","h"]: rename[c] = "high"
        elif c in ["low","l"]: rename[c] = "low"
        elif c in ["close","c","price","adj close","adjusted close"]: rename[c] = "close"
        elif c in ["volume","vol","base volume","volume usd","quote_volume"]: rename[c] = "volume"
        elif c in ["symbol","ticker","asset","coin"]: rename[c] = "symbol"

    df = df.rename(columns=rename)
    if "symbol" not in df.columns:
        df["symbol"] = symbol_from_filename(p)

    keep = [c for c in ["timestamp","open","high","low","close","volume","symbol"] if c in df.columns]
    df = df[keep].copy()

    df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce", utc=True)
    for c in ["open","high","low","close","volume"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

frames = []
for p in csv_paths:
    try:
        frames.append(load_and_standardize(p))
    except Exception as e:
        print("Skip:", p, "->", e)

raw = pd.concat(frames, ignore_index=True)
raw.shape


(342314, 7)

In [7]:
df = raw.dropna(subset=["timestamp","open","high","low","close"]).copy()
if "volume" in df.columns:
    df["volume"] = df["volume"].fillna(0)

# Sort & drop duplicates
df = df.sort_values(["symbol","timestamp"]).drop_duplicates(subset=["symbol","timestamp"])

# Hard rule violations
bad = (
    (df["high"] < df["low"]) |
    (df["open"] < df["low"]) | (df["open"] > df["high"]) |
    (df["close"] < df["low"]) | (df["close"] > df["high"]) |
    (df[["open","high","low","close"]] <= 0).any(axis=1) |
    (df["volume"] < 0)
)
print("Dropping bad rows:", int(bad.sum()))
df = df.loc[~bad].copy()

# Align to daily frequency (7 days/week)
def to_daily(g):
    g = g.set_index("timestamp").sort_index().asfreq("D")
    return g

df = (df.groupby("symbol", group_keys=False)
        .apply(to_daily)
        .reset_index())

# Remove rows with missing prices after alignment
df = df.dropna(subset=["open","high","low","close"]).copy()

print("Clean shape:", df.shape)
df.head()

Dropping bad rows: 97
Clean shape: (310896, 7)


/tmp/ipython-input-3040799768.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(to_daily)


,timestamp,open,high,low,close,volume,symbol
0,2025-10-02 00:00:00+00:00,0.230800,0.750199,0.230800,0.601250,1.397452e+09,2Z-USD
1,2025-10-03 00:00:00+00:00,0.601254,0.616968,0.511807,0.522214,5.688156e+08,2Z-USD
2,2025-10-04 00:00:00+00:00,0.522214,0.527714,0.475263,0.497500,2.359161e+08,2Z-USD
3,2025-10-05 00:00:00+00:00,0.497499,0.510207,0.482347,0.491295,2.415449e+08,2Z-USD
4,2025-10-06 00:00:00+00:00,0.491295,0.495996,0.476693,0.495729,2.873020e+08,2Z-USD


In [8]:
df = df.sort_values(['symbol','timestamp']).reset_index(drop=True)

# Log return
df['log_ret'] = df.groupby('symbol')['close'].transform(
    lambda s: np.log(s / s.shift(1))
)

# Next-day return
df['log_ret_t1'] = df.groupby('symbol')['log_ret'].shift(-1)

first_rows = df.groupby('symbol', as_index=False).head(1)
assert first_rows['log_ret'].isna().all()

df = df.replace([np.inf, -np.inf], np.nan)

df.tail()


,timestamp,open,high,low,close,volume,symbol,log_ret,log_ret_t1
310891,2024-09-19 00:00:00+00:00,7.988627,7.988627,7.988627,7.988627,0.0,ZORA-USD,0.0,0.0
310892,2024-09-20 00:00:00+00:00,7.988627,7.988627,7.988627,7.988627,0.0,ZORA-USD,0.0,0.0
310893,2024-09-21 00:00:00+00:00,7.988627,7.988627,7.988627,7.988627,0.0,ZORA-USD,0.0,0.0
310894,2024-09-22 00:00:00+00:00,7.988627,7.988627,7.988627,7.988627,0.0,ZORA-USD,0.0,0.0
310895,2024-09-23 00:00:00+00:00,7.988627,7.988627,7.988627,7.988627,0.0,ZORA-USD,0.0,NaN


In [9]:
# Rolling 95th percentile of absolute returns per symbol
def rolling_abs_q95(s, lookback=252):
    return s.abs().rolling(lookback, min_periods=100).quantile(0.95)

df['abs_q95'] = df.groupby('symbol')['log_ret'].transform(rolling_abs_q95)

# Tail-event label
df['tail_event_t1'] = (
    df['log_ret_t1'].abs() >= df.groupby('symbol')['abs_q95'].shift(1)
).astype('Int64')

# Drop rows without targets
df_model = df.dropna(subset=['log_ret', 'log_ret_t1', 'abs_q95']).copy()

print(df_model[['symbol','timestamp','log_ret','log_ret_t1','abs_q95','tail_event_t1']].head())


    symbol                 timestamp   log_ret  log_ret_t1   abs_q95  \
129  A-USD 2019-07-25 00:00:00+00:00  0.011479    0.347248  1.058644   
130  A-USD 2019-07-26 00:00:00+00:00  0.347248   -0.101958  1.057311   
131  A-USD 2019-07-27 00:00:00+00:00 -0.101958    0.327572  1.056068   
132  A-USD 2019-07-28 00:00:00+00:00  0.327572   -0.202722  1.054825   
133  A-USD 2019-07-29 00:00:00+00:00 -0.202722   -0.003537  1.053581   

     tail_event_t1  
129              0  
130              0  
131              0  
132              0  
133              0  


In [10]:
if 'df' in globals():
    base = df
elif 'df_model' in globals():
    base = df_model
elif 'out' in globals():
    base = out
else:
    raise RuntimeError("No in-memory DataFrame found. Make sure `df` (or `df_model`/`out`) exists.")

oh = base[['timestamp','symbol','volume']].copy()
oh['timestamp'] = pd.to_datetime(oh['timestamp'], errors='coerce', utc=True)
oh['volume'] = pd.to_numeric(oh['volume'], errors='coerce')

vol_profile = (
    oh.assign(is_zero = (oh['volume'].fillna(0) == 0))
      .groupby('symbol', as_index=False)
      .agg(days=('timestamp','count'),
           zero_days=('is_zero','sum'),
           zero_share=('is_zero','mean'),
           med_vol=('volume','median'),
           mean_vol=('volume','mean'))
      .sort_values(['zero_days','zero_share'], ascending=False)
)

bad_syms = vol_profile.loc[vol_profile['zero_days'] > 0, 'symbol'].tolist()

print(f"Total symbols: {vol_profile['symbol'].nunique()}")
print(f"Symbols with ANY zero-volume day: {len(bad_syms)}")
print("First 20 flagged symbols:", bad_syms[:20])

display(vol_profile.head(20))

Total symbols: 211
Symbols with ANY zero-volume day: 87
First 20 flagged symbols: ['ARB-USD', 'XCN-USD', 'A-USD', 'UBTC-USD', 'USDF-USD', 'TIA-USD', 'SKY-USD', 'JUP-USD', 'MNT-USD', 'MYX-USD', 'TRUMP-USD', 'APE-USD', 'HYPE-USD', 'NFT-USD', 'GRT-USD', 'USDE-USD', 'VSN-USD', 'TOSHI-USD', 'OSETH-USD', 'WAL-USD']


,symbol,days,zero_days,zero_share,med_vol,mean_vol
10,ARB-USD,2895,2155,0.744387,0.0,4.649761e+04
199,XCN-USD,2913,2023,0.694473,0.0,2.749279e+03
1,A-USD,2065,1282,0.620823,0.0,2.050366e+04
164,UBTC-USD,2874,1144,0.398051,19614.5,1.241600e+05
173,USDF-USD,1886,1134,0.601273,0.0,5.220439e+05
155,TIA-USD,1341,640,0.477256,123.0,1.888021e+07
138,SKY-USD,2913,552,0.189495,129953.0,7.948511e+05
87,JUP-USD,2842,507,0.178395,6734.5,4.263670e+05
103,MNT-USD,468,419,0.895299,0.0,2.947302e+04
106,MYX-USD,731,369,0.504788,0.0,2.658011e+04


In [11]:
# === Export only the cleaned OHLCV columns (timestamp, open, high, low, close, volume, symbol) ===
import os

# Pick whichever DataFrame you have in memory
base = None
if 'df' in globals():
    base = df
elif 'df_model' in globals():
    base = df_model
elif 'raw' in globals():
    base = raw
else:
    raise RuntimeError("No DataFrame found (df / df_model / raw).")

# Select ONLY the OHLCV + symbol columns
cols = ["timestamp", "open", "high", "low", "close", "volume", "symbol"]
out = base[cols].copy()

# Save
os.makedirs("/content/share", exist_ok=True)
out.to_parquet("/content/share/ohlcv_clean.parquet", index=False)
out.to_csv("/content/share/ohlcv_clean.csv", index=False)
out["symbol"].drop_duplicates().sort_values().to_csv("/content/share/symbols.txt", index=False, header=False)

print("Saved to /content/share:")
!ls -lh /content/share


Saved to /content/share:
total 46M
-rw-r--r-- 1 root root  36M Nov  1 15:32 ohlcv_clean.csv
-rw-r--r-- 1 root root  10M Nov  1 15:32 ohlcv_clean.parquet
-rw-r--r-- 1 root root 1.9K Nov  1 15:32 symbols.txt
